<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 0

Run date time (IST): 2023-07-26 08:45:53


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [4]:
# set start and end date
start_date = (datetime.date.today() + datetime.timedelta(days=-5*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')



In [5]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv')

myts = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myTransactionsReport.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

array(['5PAISA', 'AAVAS', 'AMARAJABAT', 'AMBUJACEM', 'BAJAJFINSV',
       'BAJAJHIND', 'BANDHANBNK', 'BATAINDIA', 'BAYERCROP', 'CAMS',
       'CDSL', 'CLEAN', 'DMART', 'EICHERMOT', 'EMAMILTD', 'FINEORG',
       'GICRE', 'GILLETTE', 'GLAND', 'HDFCAMC', 'HDFCBANK', 'ICICIGI',
       'INDIGOPNTS', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITBEES', 'J&KBANK',
       'JMFINANCIL', 'JUBLFOOD', 'KOTAKBANK', 'LTIM', 'LUXIND',
       'METROPOLIS', 'NIACL', 'NMDC', 'PAGEIND', 'PIDILITIND', 'PNB',
       'RAJESHEXPO', 'RELAXO', 'ROSSARI', 'SAIL', 'SRF', 'STARHEALTH',
       'SUNTV', 'TCS', 'TEAMLEASE', 'UJJIVANSFB', 'VAIBHAVGBL',
       'VALIANTORG', 'WIPRO'], dtype=object)

In [6]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='7y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),0)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in stock_ids:
      #print(stock_id)
      #current_pe = get_current_pe(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp['Prev_Close'] = tmp['Close'].shift(1)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
      tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
      tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
      tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
      tmp['Target'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Target'].values[0]
      tmp['LatestQtr'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]
      tmp['StarStock'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]
      tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]
      #tmp['Curr_PE'] = current_pe
      df_prec_dev = df_prec_dev.append(tmp)

  return df_prec_dev

df_common_features = get_common_features()

In [7]:
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

In [8]:
# portfolio stocks

def get_portfolio_features(df_common_features):

  df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])

  tmp_df = df_common_features[df_common_features['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
  print('qualified stocks: '+str(tmp_df.shape[0]))
  tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
  tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['Estimated'] = tmp_df['Target'] * tmp_df['Shares']
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['Juice Left%'] = tmp_df['Estimated P/L%'] - tmp_df['Current P/L%']
  investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
  tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['Weightage%'],0)

  return tmp_df

df_portfolio_features = get_portfolio_features(df_common_features)

def print_portfolio_stats(df_portfolio_features):
  investment = round(sum(df_portfolio_features['Avg_Cost']*df_portfolio_features['Shares']),0)
  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0)
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0)

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-investment
  curr_pnl_percentage = round((current-investment)*100/investment,2)

  estimate = round(sum(df_portfolio_features['Max']*df_portfolio_features['Shares']),0)
  est_pnl_amount = estimate-investment
  est_pnl_percentage = round((estimate-investment)*100/investment,2)

  print('Investment: ', investment)
  print('Current: ', current)
  print('Today PnL: ', today_pnl_amount)
  print('Today PnL %: ', today_pnl_percentage)
  print('Current PnL: ', curr_pnl_amount)
  print('Current PnL %: ', curr_pnl_percentage)
  print('Estimate: ', estimate)
  print('Estimate PnL: ', est_pnl_amount)
  print('Estimate PnL %: ', est_pnl_percentage)
  print('-------------------')
  print('Total profit: ',round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0))
  print('Total loss: ',round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0))

print_portfolio_stats(df_portfolio_features)

qualified stocks: 52
Investment:  3443229.0
Current:  3625125.0
Today PnL:  13458.0
Today PnL %:  0.37
Current PnL:  181896.0
Current PnL %:  5.28
Estimate:  6105003.0
Estimate PnL:  2661774.0
Estimate PnL %:  77.3
-------------------
Total profit:  223032.0
Total loss:  -41135.0


In [9]:
cols = ['Stock', 'Close', 'Dev%_200', 'RSI_14', 'Target', 'Avg_Cost', 'Shares', 'Today P/L%', 'Current P/L%', 'Estimated P/L%', 'Juice Left%', 'Weightage%', 'Risk Ind', 'LatestQtr', 'StarStock', 'Criteria', 'Strategy', 'Conviction']
df_portfolio_features = df_portfolio_features[cols]

if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_portfolio_features.to_csv('/content/drive/My Drive/data/stocks/myStocks-Portfolio-Analysis.csv', index=False)

In [10]:
df_portfolio_features.sort_values(by = 'Current P/L%', ascending=False)

,Stock,Close,Dev%_200,RSI_14,Target,Avg_Cost,Shares,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind,LatestQtr,StarStock,Criteria,Strategy,Conviction
0,5PAISA.NS,454.0,40.45,73.0,553.0,347.06,228.0,15.23,30.81,59.34,28.53,2.30,71.0,1,1.0,V40N,ATH,M
19,HDFCAMC.NS,2527.0,29.53,80.0,3518.0,1973.85,50.0,1.16,28.02,78.23,50.21,2.87,80.0,0,0.0,V40,ATH,H
17,GILLETTE.NS,5512.0,14.32,64.0,7529.0,4475.85,21.0,0.07,23.15,68.21,45.06,2.73,63.0,0,0.0,V40,ATH,H
48,UJJIVANSFB.NS,44.0,51.72,75.0,57.0,36.90,1850.0,2.33,19.24,54.47,35.23,1.98,38.0,0,1.0,V40N,ATH,M
21,ICICIGI.NS,1373.0,17.46,61.0,1613.0,1177.58,84.0,-1.08,16.60,36.98,20.38,2.87,48.0,1,1.0,V40,ATH,H
16,GICRE.NS,202.0,27.11,74.0,358.0,174.18,231.0,2.02,15.97,105.53,89.56,1.17,19.0,0,0.0,VR,2T3Y,L
45,SUNTV.NS,538.0,16.03,80.0,710.0,473.49,108.0,-0.55,13.62,49.95,36.33,1.49,20.0,0,1.0,V40N,ATH,M
4,BAJAJFINSV.NS,1649.0,11.11,70.0,1905.0,1454.93,52.0,-0.54,13.34,30.93,17.59,2.20,29.0,0,1.0,V40,ATH,H
7,BATAINDIA.NS,1701.0,6.71,63.0,2173.0,1523.62,100.0,0.95,11.64,42.62,30.98,4.42,51.0,0,0.0,V40,ATH,H
1,AAVAS.NS,1619.0,-7.15,71.0,3196.0,1453.92,37.0,0.94,11.35,119.82,108.47,1.56,18.0,0,1.0,VR,2T3Y,M


In [11]:
df_portfolio_features.sort_values(by = 'Juice Left%', ascending=True)

,Stock,Close,Dev%_200,RSI_14,Target,Avg_Cost,Shares,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind,LatestQtr,StarStock,Criteria,Strategy,Conviction
37,PIDILITIND.NS,2607.0,3.30,46.0,2902.0,2504.06,30.0,-1.70,4.11,15.89,11.78,2.18,9.0,0,1.0,V40,ATH,H
46,TCS.NS,3399.0,5.11,57.0,3847.0,3231.00,16.0,0.12,5.20,19.07,13.87,1.50,8.0,1,1.0,V40,ATH,H
13,EICHERMOT.NS,3326.0,-0.71,43.0,3850.0,3260.20,15.0,0.54,2.02,18.09,16.07,1.42,3.0,0,1.0,V40N,ATH,H
4,BAJAJFINSV.NS,1649.0,11.11,70.0,1905.0,1454.93,52.0,-0.54,13.34,30.93,17.59,2.20,29.0,0,1.0,V40,ATH,H
30,KOTAKBANK.NS,1876.0,1.76,48.0,2270.0,1955.95,26.0,-1.11,-4.09,16.06,20.15,1.48,-6.0,0,0.0,V40,BTT,L
21,ICICIGI.NS,1373.0,17.46,61.0,1613.0,1177.58,84.0,-1.08,16.60,36.98,20.38,2.87,48.0,1,1.0,V40,ATH,H
20,HDFCBANK.NS,1697.0,6.30,61.0,2050.0,1683.55,30.0,1.13,0.80,21.77,20.97,1.47,1.0,1,1.0,V40,BTT,H
0,5PAISA.NS,454.0,40.45,73.0,553.0,347.06,228.0,15.23,30.81,59.34,28.53,2.30,71.0,1,1.0,V40N,ATH,M
26,ITBEES.NS,31.0,4.40,54.0,40.0,31.49,3350.0,0.00,-1.56,27.02,28.58,3.06,-5.0,0,1.0,AR,ATH,L
5,BAJAJHIND.NS,19.0,33.93,68.0,24.0,17.15,1750.0,0.00,10.79,39.94,29.15,0.87,9.0,0,1.0,VR,ATH,M


In [12]:
df_portfolio_features.sort_values(by = 'Estimated P/L%', ascending=False)

,Stock,Close,Dev%_200,RSI_14,Target,Avg_Cost,Shares,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind,LatestQtr,StarStock,Criteria,Strategy,Conviction
18,GLAND.NS,1227.0,-13.15,72.0,4316.0,1225.87,55.0,-0.97,0.09,252.08,251.99,1.96,0.0,0,0.0,VR,2T3Y,L
50,VALIANTORG.NS,506.0,-7.42,41.0,1851.0,532.61,185.0,-1.56,-5.00,247.53,252.53,2.86,-14.0,0,0.0,VR,2T3Y,L
32,LUXIND.NS,1492.0,-1.03,51.0,4546.0,1493.72,51.0,0.74,-0.12,204.34,204.46,2.21,-0.0,0,0.0,V40N,2T3Y,M
49,VAIBHAVGBL.NS,341.0,7.91,68.0,966.0,317.48,144.0,-0.29,7.41,204.27,196.86,1.33,10.0,0,0.0,VR,2T3Y,L
6,BANDHANBNK.NS,218.0,-7.54,39.0,723.0,244.52,400.0,-0.46,-10.85,195.68,206.53,2.84,-31.0,1,0.0,VR,2T3Y,M
33,METROPOLIS.NS,1403.0,2.36,47.0,3440.0,1365.98,22.0,-0.36,2.71,151.83,149.12,0.87,2.0,0,0.0,VR,ATH,L
34,NIACL.NS,124.0,13.53,65.0,287.0,117.07,390.0,0.81,5.92,145.15,139.23,1.33,8.0,0,0.0,VR,ATH,L
47,TEAMLEASE.NS,2413.0,-1.58,51.0,5281.0,2279.93,21.0,0.92,5.84,131.63,125.79,1.39,8.0,0,1.0,V40N,2T3Y,M
22,INDIGOPNTS.NS,1502.0,16.50,68.0,3112.0,1407.73,124.0,0.60,6.70,121.07,114.37,5.07,34.0,0,1.0,V40N,2T3Y,M
1,AAVAS.NS,1619.0,-7.15,71.0,3196.0,1453.92,37.0,0.94,11.35,119.82,108.47,1.56,18.0,0,1.0,VR,2T3Y,M


In [13]:
df_portfolio_features.sort_values(by = 'Weightage%', ascending=False)

,Stock,Close,Dev%_200,RSI_14,Target,Avg_Cost,Shares,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind,LatestQtr,StarStock,Criteria,Strategy,Conviction
22,INDIGOPNTS.NS,1502.0,16.50,68.0,3112.0,1407.73,124.0,0.60,6.70,121.07,114.37,5.07,34.0,0,1.0,V40N,2T3Y,M
36,PAGEIND.NS,37196.0,-10.64,49.0,53549.0,38548.77,4.0,-0.70,-3.51,38.91,42.42,4.48,-16.0,0,1.0,V40,ATH,H
7,BATAINDIA.NS,1701.0,6.71,63.0,2173.0,1523.62,100.0,0.95,11.64,42.62,30.98,4.42,51.0,0,0.0,V40,ATH,H
24,INFY.NS,1335.0,-5.21,47.0,1848.0,1324.07,104.0,-0.15,0.83,39.57,38.74,4.00,3.0,1,1.0,V40,ATH,H
26,ITBEES.NS,31.0,4.40,54.0,40.0,31.49,3350.0,0.00,-1.56,27.02,28.58,3.06,-5.0,0,1.0,AR,ATH,L
21,ICICIGI.NS,1373.0,17.46,61.0,1613.0,1177.58,84.0,-1.08,16.60,36.98,20.38,2.87,48.0,1,1.0,V40,ATH,H
19,HDFCAMC.NS,2527.0,29.53,80.0,3518.0,1973.85,50.0,1.16,28.02,78.23,50.21,2.87,80.0,0,0.0,V40,ATH,H
50,VALIANTORG.NS,506.0,-7.42,41.0,1851.0,532.61,185.0,-1.56,-5.00,247.53,252.53,2.86,-14.0,0,0.0,VR,2T3Y,L
6,BANDHANBNK.NS,218.0,-7.54,39.0,723.0,244.52,400.0,-0.46,-10.85,195.68,206.53,2.84,-31.0,1,0.0,VR,2T3Y,M
17,GILLETTE.NS,5512.0,14.32,64.0,7529.0,4475.85,21.0,0.07,23.15,68.21,45.06,2.73,63.0,0,0.0,V40,ATH,H


In [14]:
df_portfolio_features.sort_values(by = 'Risk Ind', ascending=True)

,Stock,Close,Dev%_200,RSI_14,Target,Avg_Cost,Shares,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind,LatestQtr,StarStock,Criteria,Strategy,Conviction
6,BANDHANBNK.NS,218.0,-7.54,39.0,723.0,244.52,400.0,-0.46,-10.85,195.68,206.53,2.84,-31.0,1,0.0,VR,2T3Y,M
39,RAJESHEXPO.NS,513.0,-22.59,39.0,949.0,578.92,87.0,-0.39,-11.39,63.93,75.32,1.46,-17.0,0,1.0,V40N,ATH,M
36,PAGEIND.NS,37196.0,-10.64,49.0,53549.0,38548.77,4.0,-0.70,-3.51,38.91,42.42,4.48,-16.0,0,1.0,V40,ATH,H
50,VALIANTORG.NS,506.0,-7.42,41.0,1851.0,532.61,185.0,-1.56,-5.00,247.53,252.53,2.86,-14.0,0,0.0,VR,2T3Y,L
11,CLEAN.NS,1297.0,-10.61,31.0,2648.0,1365.11,39.0,-0.99,-4.99,93.98,98.97,1.55,-8.0,0,1.0,VR,2T3Y,L
35,NMDC.NS,112.0,-0.71,63.0,178.0,122.55,265.0,0.90,-8.61,45.25,53.86,0.94,-8.0,0,0.0,VR,ATH,M
30,KOTAKBANK.NS,1876.0,1.76,48.0,2270.0,1955.95,26.0,-1.11,-4.09,16.06,20.15,1.48,-6.0,0,0.0,V40,BTT,L
25,IRCTC.NS,619.0,-4.70,41.0,1164.0,633.05,120.0,-0.16,-2.22,83.87,86.09,2.21,-5.0,0,1.0,AR,ATH,M
26,ITBEES.NS,31.0,4.40,54.0,40.0,31.49,3350.0,0.00,-1.56,27.02,28.58,3.06,-5.0,0,1.0,AR,ATH,L
31,LTIM.NS,4869.0,4.86,43.0,7380.0,4987.59,10.0,-0.20,-2.38,47.97,50.35,1.45,-3.0,1,1.0,V200,ATH,M


In [15]:
# SIP based on portfolio
df_portfolio_features[(df_portfolio_features['Weightage%'] < 2) & (df_portfolio_features['Dev%_200'] < 0)].sort_values(by = 'RSI_14', ascending=True)

,Stock,Close,Dev%_200,RSI_14,Target,Avg_Cost,Shares,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind,LatestQtr,StarStock,Criteria,Strategy,Conviction
11,CLEAN.NS,1297.0,-10.61,31.0,2648.0,1365.11,39.0,-0.99,-4.99,93.98,98.97,1.55,-8.0,0,1.0,VR,2T3Y,L
43,SRF.NS,2164.0,-8.22,36.0,2821.0,2222.00,14.0,0.93,-2.61,26.96,29.57,0.90,-2.0,0,0.0,VR,ATH,M
39,RAJESHEXPO.NS,513.0,-22.59,39.0,949.0,578.92,87.0,-0.39,-11.39,63.93,75.32,1.46,-17.0,0,1.0,V40N,ATH,M
15,FINEORG.NS,4666.0,-9.89,41.0,7196.0,4697.43,11.0,-0.51,-0.67,53.19,53.86,1.50,-1.0,0,1.0,V40N,ATH,M
12,DMART.NS,3677.0,-2.00,43.0,5324.0,3659.46,14.0,0.25,0.48,45.49,45.01,1.49,1.0,1,1.0,AR,ATH,M
13,EICHERMOT.NS,3326.0,-0.71,43.0,3850.0,3260.20,15.0,0.54,2.02,18.09,16.07,1.42,3.0,0,1.0,V40N,ATH,H
47,TEAMLEASE.NS,2413.0,-1.58,51.0,5281.0,2279.93,21.0,0.92,5.84,131.63,125.79,1.39,8.0,0,1.0,V40N,2T3Y,M
35,NMDC.NS,112.0,-0.71,63.0,178.0,122.55,265.0,0.90,-8.61,45.25,53.86,0.94,-8.0,0,0.0,VR,ATH,M
1,AAVAS.NS,1619.0,-7.15,71.0,3196.0,1453.92,37.0,0.94,11.35,119.82,108.47,1.56,18.0,0,1.0,VR,2T3Y,M
18,GLAND.NS,1227.0,-13.15,72.0,4316.0,1225.87,55.0,-0.97,0.09,252.08,251.99,1.96,0.0,0,0.0,VR,2T3Y,L


In [16]:
#myts['Value'] = myts['Value'].str.replace(',', '').astype(float)

In [17]:
#sum(myts[(myts['Type'] == 'Buy')]['Value'])

In [18]:
#sum(myts[(myts['Type'] == 'Sell')]['Value'])